In [2]:
import types
import pandas as pd
import numpy as np
from botocore.client import Config
import ibm_boto3
def __iter__(self): return 0
# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_63a9cff2ccd745f4aac0141b03cee536 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='cEkITcKpSqIivMFetABvhrJF560Yblivtr_CN6eEn6kj',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_2 = client_63a9cff2ccd745f4aac0141b03cee536.get_object(Bucket='appleexample-donotdelete-pr-yizabrb7keakap', Key='CommViolPredUnnormalizedData.txt')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(streaming_body_2, "__iter__"): streaming_body_2.__iter__ = types.MethodType( __iter__, streaming_body_2 ) 

crime = pd.read_csv(streaming_body_2, sep=',', na_values='?')
columns_to_keep = [5, 6] + list(range(11,26)) + list(range(32, 103)) + [145]  
crime = crime.iloc[:,columns_to_keep].dropna()
X_crime = crime.iloc[:,range(0,88)]
y_crime = crime['ViolentCrimesPerPop']

In [18]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_crime,y_crime,random_state=0)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
linlasso = Lasso(alpha=2.0,max_iter=10000).fit(X_train_scaled,y_train)
# max_iter最大迭代次数
print('Lasso regression linear model coeff:\n {}'.format(linlasso.coef_))
print('Lasso regression linear model intercept (b): {:.3f}'.format(linlasso.intercept_))
print('R-squared score (training): {:.3f}'.format(linlasso.score(X_train_scaled,y_train)))
print('R-squared score (test): {:.3f}'.format(linlasso.score(X_test_scaled,y_test)))
print('Number of non-zero features: {}'.format(np.sum(linlasso.coef_ !=0)))
# linridge.coef_ !=0 返回true false矩阵
# np.sum（）将所有的true和false加起来
print('Features with non-zero weight (sorted by absolute magnitude):')
for e in sorted (list(zip(list(X_crime),linlasso.coef_)),
                key = lambda e: -abs(e[1])):
    if e[1] != 0:
        print('\t{},{:.3f}'.format(e[0],e[1]))
# key -- 主要是用来进行比较的元素，只有一个参数，具体的函数的参数就是取自于可迭代对象中，指定可迭代对象中的一个元素来进行排序。
# -abs 的符号是降序排序

Lasso regression linear model coeff:
 [    0.             0.            -0.          -168.18346054
    -0.            -0.             0.           119.6938194
     0.            -0.             0.          -169.67564456
    -0.             0.            -0.             0.
     0.             0.            -0.            -0.
     0.            -0.             0.             0.
   -57.52991966    -0.            -0.             0.
   259.32889226    -0.             0.             0.
     0.            -0.         -1188.7396867     -0.
    -0.            -0.          -231.42347299     0.
  1488.36512229     0.            -0.            -0.
    -0.             0.             0.             0.
     0.             0.            -0.             0.
    20.14419415     0.             0.             0.
     0.             0.           339.04468804     0.
     0.           459.53799903    -0.             0.
   122.69221826    -0.            91.41202242     0.
    -0.             0.             0. 

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [9]:
for alpha in [0.5,1,2,3,5,10,20,50]:
    linlasso = Lasso(alpha,max_iter=10000).fit(X_train_scaled,y_train)
    r2_train = linlasso.score(X_train_scaled,y_train)
    r2_test = linlasso.score(X_test_scaled,y_test)
    print('Alpha = {:.2f}\n\
    Features kept: {}, r-squared training: {:.2f},\
    r-squared test: {:.2f}\n'.format(alpha,np.sum(linlasso.coef_ !=0),r2_train,r2_test))

Alpha = 0.50
Features kept: 35, r-squared training: 0.65,    r-squared test: 0.58

Alpha = 1.00
Features kept: 25, r-squared training: 0.64,    r-squared test: 0.60

Alpha = 2.00
Features kept: 20, r-squared training: 0.63,    r-squared test: 0.62

Alpha = 3.00
Features kept: 17, r-squared training: 0.62,    r-squared test: 0.63

Alpha = 5.00
Features kept: 12, r-squared training: 0.60,    r-squared test: 0.61

Alpha = 10.00
Features kept: 6, r-squared training: 0.57,    r-squared test: 0.58

Alpha = 20.00
Features kept: 2, r-squared training: 0.51,    r-squared test: 0.50

Alpha = 50.00
Features kept: 1, r-squared training: 0.31,    r-squared test: 0.30



In [21]:
list(X_crime)

['population',
 'householdsize',
 'agePct12t21',
 'agePct12t29',
 'agePct16t24',
 'agePct65up',
 'numbUrban',
 'pctUrban',
 'medIncome',
 'pctWWage',
 'pctWFarmSelf',
 'pctWInvInc',
 'pctWSocSec',
 'pctWPubAsst',
 'pctWRetire',
 'medFamInc',
 'perCapInc',
 'NumUnderPov',
 'PctPopUnderPov',
 'PctLess9thGrade',
 'PctNotHSGrad',
 'PctBSorMore',
 'PctUnemployed',
 'PctEmploy',
 'PctEmplManu',
 'PctEmplProfServ',
 'PctOccupManu',
 'PctOccupMgmtProf',
 'MalePctDivorce',
 'MalePctNevMarr',
 'FemalePctDiv',
 'TotalPctDiv',
 'PersPerFam',
 'PctFam2Par',
 'PctKids2Par',
 'PctYoungKids2Par',
 'PctTeen2Par',
 'PctWorkMomYoungKids',
 'PctWorkMom',
 'NumKidsBornNeverMar',
 'PctKidsBornNeverMar',
 'NumImmig',
 'PctImmigRecent',
 'PctImmigRec5',
 'PctImmigRec8',
 'PctImmigRec10',
 'PctRecentImmig',
 'PctRecImmig5',
 'PctRecImmig8',
 'PctRecImmig10',
 'PctSpeakEnglOnly',
 'PctNotSpeakEnglWell',
 'PctLargHouseFam',
 'PctLargHouseOccup',
 'PersPerOccupHous',
 'PersPerOwnOccHous',
 'PersPerRentOccHous',
 

In [19]:
X_crime

,population,householdsize,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,...,MedRentPctHousInc,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumInShelters,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85
0,11980,3.10,12.47,21.44,10.93,11.33,11980,100.00,75122,89.24,...,23.8,21.1,14.0,11,0,10.66,53.72,65.29,78.09,89.14
1,23123,2.82,11.01,21.30,10.48,17.18,23123,100.00,47917,78.99,...,27.6,20.7,12.5,0,0,8.30,77.17,71.27,90.22,96.12
2,29344,2.43,11.36,25.88,11.01,10.28,29344,100.00,35669,82.00,...,24.1,21.7,11.6,16,0,5.00,44.77,36.60,61.26,82.85
3,16656,2.40,12.55,25.20,12.19,17.57,0,0.00,20580,68.15,...,28.7,20.6,14.5,0,0,2.04,88.71,56.70,90.17,96.24
5,140494,2.45,18.09,32.89,20.04,13.26,140494,100.00,21577,75.78,...,26.4,17.3,11.7,327,4,1.49,64.35,42.29,70.61,85.66
6,28700,2.60,11.17,27.41,12.76,14.42,28700,100.00,42805,79.47,...,24.4,20.8,12.5,0,0,9.19,77.30,63.45,82.23,93.53
7,59459,2.45,15.31,27.93,14.78,14.60,59449,100.00,23221,71.60,...,26.3,15.1,12.2,21,0,0.87,73.70,54.85,85.55,91.51
8,74111,2.46,16.64,35.16,20.33,8.58,74115,100.00,25326,83.69,...,25.2,20.7,12.8,125,15,1.99,58.82,40.72,67.97,81.39
9,103590,2.62,19.88,34.55,21.62,13.12,103590,100.00,17852,74.20,...,29.6,19.4,13.0,43,4,4.63,75.59,42.33,74.05,92.12
10,31601,2.54,15.73,28.57,15.16,14.26,31596,100.00,24763,73.92,...,23.8,17.1,12.9,1,0,2.97,65.73,44.95,74.82,88.66
